# Generate Text to Image

Further convert images to GIF

In [ ]:
!pip install split-image
!pip install min-dalle

In [ ]:
from IPython.display import display, update_display # we use this to show images.
import torch
from min_dalle import MinDalle # dalle mini library
from PIL import Image
from split_image import split_image
import glob
import imageio
import os

In [ ]:
try:
  os.makedirs('Temp_grid_image')
except:
  pass
try:
  os.makedirs('Split_grid')
except:
  pass
try:
  os.makedirs('Output_GIF')
except:
  pass

In [ ]:
# import model text to image model
model = MinDalle(
    dtype=torch.float16, # float16 is faster than float32, though float32 will generate better images
    device = None, # set this to None if you don't have GPU else cuda
    is_mega = True, # set it to False if you don't want to use large model
    is_reusable=True
)

using device cuda
downloading tokenizer params
intializing TextTokenizer
downloading encoder params
initializing DalleBartEncoder
downloading decoder params
initializing DalleBartDecoder
downloading detokenizer params
initializing VQGanDetokenizer


In [ ]:
# Generate Text to GIF
def generate_text_to_img(model, text):
    # parameters
    text = text
    progressive_outputs = True # It will keep generating and improving the images
    seamless = False # set it to True if you want to mix all images
    grid_size = 2 # generates 3x3 grid of images
    temperature = 9
    supercondition_factor = 32
    top_k = 256
    
    # generate image
    image_stream = model.generate_image_stream(
    text=text,
    seed=-1,
    grid_size = grid_size,
    progressive_outputs = progressive_outputs,
    is_seamless = seamless,
    temperature=temperature,
    top_k = int(top_k),
    supercondition_factor = supercondition_factor,
    is_verbose=False
    )
    
    # save image
    for image in image_stream:
        image.save('/content/Temp_grid_image/img.png')
        
    # split grid image into 3x3
    split_image("Temp_grid_image/img.png",rows = 3,cols = 3,should_square = True, should_cleanup = False,output_dir = "Split_grid/")
    
    # generte GIF
    images = []
    for filepath in glob.iglob('Split_grid/*.png'):
      if "squared" not in filepath:
        images.append(imageio.imread(filepath))
    
    imageio.mimsave('Output_GIF/out.gif', images,format='GIF',duration=0.7) # generate gif
    
    return "Pass"

In [ ]:
%%time
text = "produce a paint image generated by AI"

generate_text_to_img(model,text)

Resizing image to a square...
Determining background color...
Background color is... (0, 0, 0, 255)
Exporting resized image...
Exporting image tile: Split_grid/img_0.png
Exporting image tile: Split_grid/img_1.png
Exporting image tile: Split_grid/img_2.png
Exporting image tile: Split_grid/img_3.png
Exporting image tile: Split_grid/img_4.png
Exporting image tile: Split_grid/img_5.png
Exporting image tile: Split_grid/img_6.png
Exporting image tile: Split_grid/img_7.png
Exporting image tile: Split_grid/img_8.png
CPU times: user 15.6 s, sys: 247 ms, total: 15.8 s
Wall time: 15.7 s


'Pass'